## CVAT 檔案下載
- 請先至 Google 表單手動下載[標註工作分配](https://docs.google.com/spreadsheets/d/108W2SbwEj4654nnM5G7KbMiW0pKutBXfLbV2Dxsb3jo/edit?gid=1594279948#gid=1594279948)
- 並將檔案放置 annotation 資料夾，建議將檔案加上日期後綴

In [ ]:
credential = "jeffhong824@gmail.com"
password = "Jeff840606"
process_type = "yolo"
# process_type = "cvat_for_video"
target_dir = f"../dataset/{process_type}_type/project"
# 若資料夾不存在請自行創立
save_img = False
headless = False

In [22]:
from time import sleep
from tqdm import tqdm
import time
import shutil

from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver import Chrome
import pandas as pd

## 標註 excel 統計

In [20]:
file_path = "../annotation/標註工作分配_20251022.xlsx"
sheets_to_process = ["DA1", "DA2", "DA3", "DA4_新竹", "DA5", "DA6", "DA7_新竹", "內視鏡1", "內視鏡2", "內視鏡3", "內視鏡4", "內視鏡5"]

# 初始化名單
ready_to_download = []  # 準備下載的 ID
not_ready_to_download = []  # 尚未準備下載的 ID

# 讀取每個指定的 sheet
for sheet in sheets_to_process:
    count_ready = 0
    count_not_ready = 0
    count_annotation_error = 0
    df = pd.read_excel(file_path, sheet_name=sheet, dtype={"cl_grade": str})  # 確保 cl_grade 是字串
    # print(sheet)

    # 假設第一欄是 ID，且 "凡雅備註" 和 "cl_grade" 是關鍵欄位
    id_column = df.columns[0]  # 取得第一欄的名稱
    
    for _, row in df.iterrows():
        id_value = row[id_column]
        annotation = str(row.get("完成", ""))
        # note = str(row.get("凡雅備註", ""))
        cl_grade = str(row.get("cl_grade", "")).strip()
        
        # 檢查 ID 是否為數字型態
        # if pd.notna(id_value) and isinstance(id_value, (int, float)):
            # id_value = int(id_value)  # 轉為整數
        
        if str(id_value).isdigit() and len(str(id_value)) == 6:
            id_value_re = "0" + str(int(id_value))
        elif str(id_value).isdigit() and len(str(id_value)) == 7:
            id_value_re = str(int(id_value))
        else:
            id_value_re = str(id_value)

        if sheet.find("DA")!=-1:
            id_value_re = "DA" + id_value_re
        elif sheet.find("內視鏡")!=-1:
            id_value_re = "ES" + id_value_re
        

        # 判斷條件
        if ( annotation.find("V")!=-1 or annotation.find("v")!=-1 ): # and cl_grade.isdigit(): # and int(cl_grade) in range(1, 7): # note == "計算距離沒問題"
            ready_to_download.append(id_value_re)
            count_ready += 1
        elif ( str(annotation)!="nan" and annotation.find("V")==-1 and annotation.find("v")==-1 ): # and cl_grade.isdigit(): # and int(cl_grade) in range(1, 7): # note == "計算距離沒問題"

            not_ready_to_download.append(id_value_re)
            count_annotation_error += 1
        else:
            not_ready_to_download.append(id_value_re)
            count_not_ready += 1

    print(sheet, "count_ready", count_ready, "count_annotation_error", count_annotation_error, "count_not_ready", count_not_ready)

# 輸出結果
print("準備下載名單:", len(ready_to_download), ready_to_download)
print("尚未準備下載名單:", len(not_ready_to_download), not_ready_to_download)


DA1 count_ready 71 count_annotation_error 6 count_not_ready 3
DA2 count_ready 75 count_annotation_error 0 count_not_ready 0
DA3 count_ready 38 count_annotation_error 0 count_not_ready 0
DA4_新竹 count_ready 55 count_annotation_error 0 count_not_ready 11
DA5 count_ready 2 count_annotation_error 0 count_not_ready 43
DA6 count_ready 22 count_annotation_error 0 count_not_ready 49
DA7_新竹 count_ready 0 count_annotation_error 0 count_not_ready 23
內視鏡1 count_ready 145 count_annotation_error 2 count_not_ready 2
內視鏡2 count_ready 94 count_annotation_error 3 count_not_ready 120
內視鏡3 count_ready 54 count_annotation_error 4 count_not_ready 12
內視鏡4 count_ready 102 count_annotation_error 0 count_not_ready 3
內視鏡5 count_ready 26 count_annotation_error 0 count_not_ready 40
準備下載名單: 684 ['DA2249663', 'DA2317516', 'DA2333025', 'DA2661769', 'DA2691356', 'DA3106245', 'DA3262442', 'DA3310343', 'DA3376168', 'DA3488514', 'DA3709145', 'DA3946795', 'DA3966152', 'DA4002122', 'DA4030655', 'DA4077072', 'DA4099071', 'DA

## 已處理 path 統計

In [38]:
import os

# 初始化名單
da_items = []

# 瀏覽資料夾內容
for item in os.listdir(target_dir):
    da_items.append(str(item).replace(".zip", ""))

# 輸出結果
print("整理後的名單:", len(da_items), da_items)


整理後的名單: 7 ['DA2249663', 'DA2317516', 'DA2333025', 'DA2661769', 'DA2691356', 'DA3106245', 'DA3262442']


In [39]:
for da in da_items:
    if da not in ready_to_download :
        print("不在名單內:", da)

## 準備下載名單

In [40]:
# 使用集合運算去除重複項
filtered_list = list(set(ready_to_download) - set(da_items))

# 保持原本的順序
filtered_list = [item for item in ready_to_download if item not in set(da_items)]
print("要下載的名單:", len(filtered_list), filtered_list)


要下載的名單: 677 ['DA3310343', 'DA3376168', 'DA3488514', 'DA3709145', 'DA3946795', 'DA3966152', 'DA4002122', 'DA4030655', 'DA4077072', 'DA4099071', 'DA4225273', 'DA4424313', 'DA4456080', 'DA4494352', 'DA4531952', 'DA5135292', 'DA5646048', 'DA5652785', 'DA5658925', 'DA5700232', 'DA5810934', 'DA6033260', 'DA6192696', 'DA6261477', 'DA6262262', 'DA6486444', 'DA6561173', 'DA6615914', 'DA6645864', 'DA6669644', 'DA6817030', 'DA6831268', 'DA6921084', 'DA6956411', 'DA7037727', 'DA7052421', 'DA7137314', 'DA7146063', 'DA7146476', 'DA7146653', 'DA7146655', 'DA7154898', 'DA7155348', 'DA7156856', 'DA7157082', 'DA7158396', 'DA7160836', 'DA7162270', 'DA7185295', 'DA2330478', 'DA2735247', 'DA3039600', 'DA3097072', 'DA3200270', 'DA3979187', 'DA5343641', 'DA6482830', 'DA6913987', 'DA8045956', 'DA3967814', 'DA6350366', 'DA6921735', 'DA7071617', 'DA7134812', 'DA587648.0', 'DA1345365.0', 'DA1968870.0', 'DA2107336.0', 'DA2265988.0', 'DA2531513.0', 'DA2593149.0', 'DA2647824.0', 'DA2723789.0', 'DA2767782.0', 'DA276

## 下載 

In [ ]:
chrome_options = Options()
if headless:
    chrome_options.add_argument('--headless')
driver = Chrome(options=chrome_options)

In [ ]:
download_dir = "C:\\Users\\user\\Downloads"  # 請改成實際下載資料夾
timeout = 120  # 最多等待 120 秒

In [ ]:
for patient_ID in tqdm(filtered_list[:50], desc="Downloading datasets"):

    ## login
    driver.get('https://aiailab.synology.me/projects')
    try_login = True
    stop_time = 10
    stop_time_count = 0
    while try_login:
        try:
            driver.find_element(By.XPATH, '//*[@id="credential"]').send_keys(credential)
            try_login = False
        except:
            try_login = True
            stop_time_count += 1
            if stop_time_count >= stop_time:
                try_login = False
    if stop_time_count < stop_time:
        driver.find_element(By.XPATH, '//*[@id="password"]').send_keys(password)
        driver.find_element(By.XPATH, '//*[@id="root"]/section/section/main/div/div[2]/div/div/div/form/div[3]/div/div/div/button').click()


    ## find patient ID
    try_search = True
    while try_search:
        try:
            driver.find_element(By.XPATH, '//*[@id="root"]/section/main/div/div[1]/div/div[1]/span/span/input').send_keys(patient_ID)
            try_search = False
        except:
            try_search = True

    try_click = True
    while try_click:
        try:
            driver.find_element(By.XPATH, '//*[@id="root"]/section/main/div/div[1]/div/div[1]/span/span/span/button').click()
            try_click = False
        except:
            try_click = True

    ## Export dataset
    try_point_button = True
    while try_point_button:
        try:
            driver.find_element(By.XPATH, '//*[@id="root"]/section/main/div/div[2]/div/div/div[1]/div[2]/div/div/div[2]/div/div[2]/button').click()
            try_point_button = False
        except:
            try_point_button = True

    try_export_button = True
    while try_export_button:
        try:
            driver.find_element(By.XPATH, '/html/body/div[3]/div/div/ul/li[1]/span').click()
            try_export_button = False
        except:
            try_export_button = True
    
    try_export_dataset_button = True
    while try_export_dataset_button:
        try:
            driver.find_element(By.XPATH, '//*[@id="Export dataset"]/div[1]/div[2]/div/div/div').click()
            try_export_dataset_button = False
        except:
            try_export_dataset_button = True

    
    try_select_button = True
    while try_select_button:
        try:
            if process_type[:4] == "yolo":
                driver.find_element(By.XPATH, '/html/body/div[5]/div/div/div/div[2]/div/div/div/div[24]').click()
            elif process_type == "cvat_for_video":
                driver.find_element(By.XPATH, '/html/body/div[5]/div/div/div/div[2]/div/div/div/div[6]/div').click()
            try_select_button = False
        except:
            try_select_button = True

    ## save images
    if save_img == True:
        driver.find_element(By.XPATH, '//*[@id="Export dataset_saveImages"]').click()

    ## dataset name
    driver.find_element(By.XPATH, '//*[@id="Export dataset_customName"]').send_keys(patient_ID)

    ## OK
    driver.find_element(By.XPATH, '/html/body/div[4]/div/div[2]/div/div[2]/div[3]/button[2]').click()

    expected_filename = f"{patient_ID}.zip"
    downloaded = False
    start_time = time.time()
    
    while not downloaded and time.time() - start_time < timeout:
        for filename in os.listdir(download_dir):
            if filename.endswith(".zip") and expected_filename in filename:
                source_path = os.path.join(download_dir, filename)
                dest_path = os.path.join(target_dir, filename)

                # 移動檔案
                shutil.move(source_path, dest_path)
                # print(f"已移動: {filename} 至 {target_dir}")
                downloaded = True
                break
        if not downloaded:
            time.sleep(2)  # 每 2 秒檢查一次

    if not downloaded:
        print(f"[警告] 等待 {timeout} 秒後仍未找到檔案 {expected_filename}，跳過此 ID。")


KeyboardInterrupt: 